In [5]:
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import matplotlib
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
data_path = os.chdir("C:\\Users\\NoMad\\PycharmProjects\\SANEM\\movements")

In [21]:
# другой вариант таблицы, где 25 - число движений, 24 - число пространственных переменных, n - число моментов, потребовавшихся для записи движения
# (25, 24, n)
x_train = np.array(pd.read_csv('m01_s01_e01_positions.txt', sep=','))
x_train = np.array([x_train[:, 42:].transpose()])
print(x_train.shape)

for i in range(1,7):
    for j in range(2,6):
        data1 = np.array(pd.read_csv(f"m0{i}_s0{j}_e01_positions.txt", sep=','))
        data1 = np.array([data1[:, 42:].transpose()])

        if(x_train.shape[2]>data1.shape[2]):
            x_train = x_train[:,:,:data1.shape[2]]
            x_train = np.append(x_train, data1, axis=0)
        elif(x_train.shape[2]<data1.shape[2]):
            data1 = data1[:,:,:x_train.shape[2]]
            x_train = np.append(x_train, data1, axis=0)
        elif(x_train.shape[2]==data1.shape[2]):
            x_train = np.append(x_train, data1, axis=0)

print(x_train.shape)

(1, 24, 72)
(25, 24, 51)


In [22]:
x_train = x_train.reshape(x_train.shape[0], 24, x_train.shape[2], 1)
x_train = x_train.astype('float32')
print(x_train.shape)

(25, 24, 51, 1)


In [9]:
# проверочные данные:
predict_move_2 = np.array(pd.read_csv('m02_s01_e01_positions.txt', sep=','))
predict_move_2 = np.array([predict_move_2[:51, 42:].transpose()])
print(predict_move_2.shape)
predict_move_2 = predict_move_2.reshape(predict_move_2.shape[0], 24, predict_move_2.shape[2], 1)
predict_move_2 = predict_move_2.astype('float32')
print("predict_move_2 = ",predict_move_2.shape)

predict_move_4 = np.array(pd.read_csv('m04_s01_e01_positions.txt', sep=','))
predict_move_4 = np.array([predict_move_4[:51, 42:].transpose()])
print(predict_move_4.shape)
predict_move_4 = predict_move_4.reshape(predict_move_4.shape[0], 24, predict_move_4.shape[2], 1)
predict_move_4 = predict_move_4.astype('float32')
print("predict_move_4 = ",predict_move_4.shape)

predict_move_5 = np.array(pd.read_csv('m05_s01_e01_positions.txt', sep=','))
predict_move_5 = np.array([predict_move_5[:51, 42:].transpose()])
print(predict_move_5.shape)
predict_move_5 = predict_move_5.reshape(predict_move_5.shape[0], 24, predict_move_5.shape[2], 1)
predict_move_5 = predict_move_5.astype('float32')
print("predict_move_5 = ",predict_move_5.shape)

predict_move_6 = np.array(pd.read_csv('m06_s01_e05_positions.txt', sep=','))
predict_move_6 = np.array([predict_move_6[:51, 42:].transpose()])
print(predict_move_6.shape)
predict_move_6 = predict_move_6.reshape(predict_move_6.shape[0], 24, predict_move_6.shape[2], 1)
predict_move_6 = predict_move_6.astype('float32')
print("predict_move_6 = ",predict_move_6.shape)

(1, 24, 51)
predict_move_2 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_4 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_5 =  (1, 24, 51, 1)
(1, 24, 51)
predict_move_6 =  (1, 24, 51, 1)


In [11]:
# выходной набор данных. Столбец с единицей соответствует номеру соответствующего движения (движений всего 6) 
# y_train = [[1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0], [1,0,0,0,0,0],
#            [0,1,0,0,0,0], [0,1,0,0,0,0], [0,1,0,0,0,0], [0,1,0,0,0,0],
#            [0,0,1,0,0,0], [0,0,1,0,0,0], [0,0,1,0,0,0], [0,0,1,0,0,0],
#            [0,0,0,1,0,0], [0,0,0,1,0,0], [0,0,0,1,0,0], [0,0,0,1,0,0],
#            [0,0,0,0,1,0], [0,0,0,0,1,0], [0,0,0,0,1,0], [0,0,0,0,1,0],
#            [0,0,0,0,0,1], [0,0,0,0,0,1], [0,0,0,0,0,1]]

y_train = [0,0,0,0,0,1,1,1,1,2,2,2,2,3,3,3,3,4,4,4,4,5,5,5,5]
y_train = np.array(y_train, dtype=np.float32)
y_train = keras.utils.to_categorical(y_train)
print(y_train[:6])

[[1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]


In [23]:
# другой вариант нейросети (сверточная)
input = keras.Input(shape=(24,x_train.shape[2],1))
inputs = layers.Convolution2D(25, input_shape=(24,), kernel_size=(3,1), activation="relu")(input)
hidden1 = layers.Convolution2D(25, kernel_size=(3,1), activation="softmax")(inputs)
hidden1 = layers.Dropout(0.2)(hidden1)
h1 = layers.Flatten()(hidden1)
outputs = layers.Dense(6, activation="sigmoid")(h1)

seq_model = keras.Model(inputs=input, outputs=outputs, name="sequential_or_what")
seq_model.summary()

Model: "sequential_or_what"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 24, 51, 1)]       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 22, 51, 25)        100       
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 51, 25)        1900      
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 51, 25)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 25500)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 153006    
Total params: 155,006
Trainable params: 155,006
Non-trainable params: 0
__________________________________________

In [24]:
seq_model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [25]:
seq_model.fit(x_train,y_train,epochs=100,batch_size=2)

Epoch 1/100
13/13 [==============================] - 0s 8ms/step - loss: 0.6106 - accuracy: 0.0800
Epoch 2/100
13/13 [==============================] - 0s 8ms/step - loss: 0.4941 - accuracy: 0.1200
Epoch 3/100
13/13 [==============================] - 0s 8ms/step - loss: 0.5024 - accuracy: 0.0800
Epoch 4/100
13/13 [==============================] - 0s 8ms/step - loss: 0.4694 - accuracy: 0.1600
Epoch 5/100
13/13 [==============================] - 0s 8ms/step - loss: 0.4494 - accuracy: 0.2800
Epoch 6/100
13/13 [==============================] - 0s 8ms/step - loss: 0.4713 - accuracy: 0.2800
Epoch 7/100
13/13 [==============================] - 0s 8ms/step - loss: 0.4554 - accuracy: 0.2400
Epoch 8/100
13/13 [==============================] - 0s 8ms/step - loss: 0.4014 - accuracy: 0.3200
Epoch 9/100
13/13 [==============================] - 0s 8ms/step - loss: 0.4148 - accuracy: 0.3200
Epoch 10/100
13/13 [==============================] - 0s 8ms/step - loss: 0.3900 - accuracy: 0.4400
Epoch 11/

13/13 [==============================] - 0s 8ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 83/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0099 - accuracy: 1.0000
Epoch 84/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0090 - accuracy: 1.0000
Epoch 85/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0096 - accuracy: 1.0000
Epoch 86/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0095 - accuracy: 1.0000
Epoch 87/100
13/13 [==============================] - 0s 7ms/step - loss: 0.0092 - accuracy: 1.0000
Epoch 88/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0087 - accuracy: 1.0000
Epoch 89/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 90/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0088 - accuracy: 1.0000
Epoch 91/100
13/13 [==============================] - 0s 8ms/step - loss: 0.0071 - accuracy: 1.0000
Epoch 92/100


In [67]:
predictArr = seq_model.predict(predict_move_2)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[3.1037456e-03 9.1514987e-01 1.7866507e-04 4.9094233e-04 1.2775529e-03
 1.5232771e-03]
91.51 % this is move №  2


In [69]:
predictArr = seq_model.predict(predict_move_4)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[4.1219688e-04 9.6837291e-05 1.6512241e-03 9.8281801e-01 5.5855144e-02
 1.1280424e-05]
98.28 % this is move №  4


In [70]:
predictArr = seq_model.predict(predict_move_5)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[1.1729021e-02 1.9642316e-04 2.3820330e-04 3.7410727e-03 9.8527676e-01
 5.0973076e-06]
98.53 % this is move №  5


In [72]:
predictArr = seq_model.predict(predict_move_6)
print(predictArr[0])

maxPercentage = 0
for i in enumerate(predictArr[0]):
    if(i[1]>maxPercentage):
        maxPercentage=i[1]
        move=i[0]+1     
print(round(maxPercentage*100, 2),"% this is move № ", move)

[3.9290026e-05 9.9408722e-01 3.6579502e-05 5.2649714e-04 5.5230624e-04
 5.4420933e-02]
99.41 % this is move №  2
